In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import neptune # for logging progress and results
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from attrs import define, field, fields, Factory

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType

from pyphocorehelpers.print_helpers import generate_html_string

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions

import pyphoplacecellanalysis.External.pyqtgraph as pg
# 'time_bin_indices': valid_time_bin_indicies, 'posterior_to_pf_mean_surprise': one_left_out_posterior_to_pf_surprises_mean, 'posterior_to_scrambled_pf_mean_surprise': one_left_out_posterior_to_scrambled_pf_surprises_mean}
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_expected_vs_observed_firing_rates
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_surprise_difference_plot

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import _update_pipeline_missing_preprocessing_parameters

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots

from scripts.run_BatchAnalysis import post_compute_validate, _on_complete_success_execution_session

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

good_contexts_list = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54')
]

from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass




In [3]:
%pdb off
# %%viztracer
# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE


local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
# saving_mode = PipelineSavingScheme.SKIP_SAVING
# force_reload = False

# Force write:
# saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
force_reload = True


## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.


    # ==================================================================================================================== #
    # Load Pipeline                                                                                                        #
    # ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation',  # AL:WAYS OFF
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation', # AL:WAYS OFF
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation' # AL:WAYS OFF
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding' # AL:WAYS OFF
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'


if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        print(f'cannot load global results: {e}')

# 6m 1.1s
# 12m 15.6s


Automatic pdb calling has been turned OFF
basedir: /media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-16-25
Skipping loading from pickled file because force_reload == True.


/home/halechr/repo/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:140: UserWarning: WARNING: Optional File: /media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-16-25/2006-6-08_21-16-25.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-16-25/2006-6-08_21-16-25.epochs_info.mat... done.
Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-16-25/2006-6-08_21-16-25.position_info.mat... done.
Loading matlab import file results : /media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-16-25/2006-6-08_21-16-25.spikes.mat... done.
Failure loading .position.npy. Must recompute.

Saving updated position results results : /media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-16-25/2006-6-08_21-16-25.position.npy... 2006-6-08_21-16-25.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.
	 Saving updated interpolated spike position results results : /media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-16-25/2006-6-08_21-16-25.interpolated_spike_positions.npy... 2006-6-08_21-16-25.interpolated_spike_positions.npy save

/home/halechr/repo/NeuroPy/neuropy/core/laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
/home/halechr/repo/NeuroPy/neuropy/core/laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
/home/halechr/repo/NeuroPy/neuropy/core/laps.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


Applying session filter named "maze2"...
Constraining to epoch with times (start: 722.653951405664, end: 1201.0839364149142)
computing neurons mua for session...

Applying session filter named "maze"...
Constraining to epoch with times (start: 0.0, end: 1201.0839364149142)
computing neurons mua for session...



/home/halechr/repo/NeuroPy/neuropy/core/laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
/home/halechr/repo/NeuroPy/neuropy/core/laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
/home/halechr/repo/NeuroPy/neuropy/core/laps.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
due to includelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (17.01858788173554, 250.2171441367766)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((17.01858788173554, 250.2171441367766), (135.66814125966783, 154.75073313142283))
	 done.
	 spikes_df[time_variable_name]: (19348,) should be less than time_window_edges: (20071,)!
	 spikes_df[time_variable_name]: (19348,) should be less than time_window_edges: (20071,)!


/home/halechr/repo/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/Analysis/Decoder/reconstruction.py:363: RuntimeWarning: divide by zero encountered in divide
  return C * np.exp(numerator/denominator)


two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (20269,)
updating computation_results...
done.
due to includelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (17.01858788173554, 250.2171441367766)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((17.01858788173554, 250.2171441367766), (135.66814125966783, 154.75073313142283))
	 done.
two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (12894,)
updating computation_results...
done.
due to includelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (17.01858788173554, 250.2171441367766)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((17.01858788173554, 250.2171441367766), (135.66814125966783, 154.75073313142283))


In [ ]:
%matplotlib qt
from neuropy.utils.matplotlib_helpers import interactive_select_grid_bin_bounds_2D
# fig, ax, rect_selector, set_extents = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True)

grid_bin_bounds = interactive_select_grid_bin_bounds_2D(curr_active_pipeline, epoch_name='maze', should_block_for_input=True, should_apply_updates_to_pipeline=False)
print(f'grid_bin_bounds: {grid_bin_bounds}')
print(f"Add this to `specific_session_override_dict`:\n\n{curr_active_pipeline.get_session_context().get_initialization_code_string()}:dict(grid_bin_bounds=({(grid_bin_bounds[0], grid_bin_bounds[1]), (grid_bin_bounds[2], grid_bin_bounds[3])})),\n")

In [4]:
## Post Compute Validate 2023-05-16:
from scripts.run_BatchAnalysis import post_compute_validate

post_compute_validate(curr_active_pipeline)

were pipeline preprocessing parameters missing and updated?: False


In [5]:
curr_active_pipeline.reload_default_computation_functions()
extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'] # do only specifiedl , 'long_short_rate_remapping'
force_recompute_global = force_reload
# force_recompute_global = True
# extended_computations_include_includelist=['long_short_post_decoding'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if len(newly_computed_values) > 0:
	print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
	try:
		# curr_active_pipeline.global_computation_results.persist_time = datetime.now()
		# Try to write out the global computation function results:
		curr_active_pipeline.save_global_computation_results()
	except Exception as e:
		print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
		print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
	print(f'no changes in global results.')
# 10m 29.5s

included includelist is specified: ['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis missing.
	 Recomputing jonathan_firing_rate_analysis...
	 done.
long_short_fr_indicies_analyses missing.
	 Recomputing long_short_fr_indicies_analyses...
	 done.
long_short_decoding_analyses missing.
	 Recomputing long_short_decoding_analyses...
sane_midpoint_x: 135.71178651132138, hardcoded_track_midpoint_x: None, track_min_max_x: (15.182688599114876, 256.24088442352786)
hardcoded_track_midpoint_x is None, falling back to sane_midpoint_x... 135.71178651132138
desc_crossings_x: (20,), asc_crossings_x: (20,)
setting new computation epochs because laps changed.
using self.config.grid_bin_bounds_1D: (17.01858788173554, 250.2171441367766)
using self.config.grid_bin_bounds: ((17.01

In [6]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)


https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/
https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-100
shared_aclus: [2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102]
long_only_aclus: [19, 23, 36, 54, 60, 78, 101, 103]
short_only_aclus: [8, 82]
shared_aclus: [2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102]
long_only_aclus: [19, 23, 36, 

### Pipeline Loading/Saving [Optional]

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object
# PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell
# TypeError: cannot pickle 'PyQt5.QtCore.pyqtSignal' object

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)

In [ ]:
curr_active_pipeline.save_global_computation_results() # PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell

In [ ]:
curr_active_pipeline.load_pickled_global_computation_results()

In [ ]:
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
curr_active_pipeline.save_global_computation_results()

# Get output files:

In [ ]:
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatBaseRegisteredClass
from neuropy.core.session.KnownDataSessionTypeProperties import KnownDataSessionTypeProperties
from neuropy.core.session.dataSession import DataSession
from neuropy.core.session.Formats.SessionSpecifications import SessionFolderSpec, SessionFileSpec

# 'ripple_df.pkl', .ripple.npy'

# files_list = [curr_active_pipeline.sess.filePrefix.with_suffix(a_suffix) for a_suffix in session_folder_output_file_suffix_list]
# files_list

"""
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.interpolated_spike_positions.npy,"2023-06-07 01:03:36",35290406
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.mua.npy,"2023-06-07 01:03:37",7766554
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.pbe.npy,"2023-06-07 01:03:37",10902
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.position.npy,"2023-06-07 01:03:36",2446247
"""

session_name = curr_active_pipeline.session_name
print(f'session_name: {session_name}')
output_folder_spec: SessionFolderSpec = SessionFolderSpec(required=[
        SessionFileSpec('loadedSessPickle.pkl', session_name, 'The primary .pkl file after computing the session', None),
        SessionFileSpec('{}.interpolated_spike_positions.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('{}.mua.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('{}.pbe.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('{}.position.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('output/global_computation_results.pkl', session_name, 'The global_computation_results computed for the pipeline.', None),
    ],
    optional=[
        SessionFileSpec('output/leave_one_out_results_short.pkl', session_name, 'Optional caching file.', None),
        SessionFileSpec('output/leave_one_out_surprise_results_long.pkl', session_name, 'Optional caching file', None),
        SessionFileSpec('ripple_df.pkl', session_name, 'SWR ripples computed by an external script from another repo.', None),
        SessionFileSpec('{}.ripple.npy', session_name, 'Ripples computed by this repo and cached to the directory.', None),
    ]
)


is_resolved, resolved_required_filespecs_dict, resolved_optional_filespecs_dict = output_folder_spec.validate(curr_active_pipeline.sess.basepath)

if is_resolved:
    print(f'resolved_required_filespecs_dict: {resolved_required_filespecs_dict}')
    

# out_dict = {}
# out_dict['required_files'] = [str(a_path) for a_path in resolved_required_filespecs_dict.keys() if a_path.resolve().exists()]
# out_dict['optional_files'] = [str(a_path) for a_path in resolved_optional_filespecs_dict.keys() if a_path.resolve().exists()]
# out_dict

# Get the list of strings for all output files that actually exist:
all_output_file_paths: list[str] = [str(a_path) for a_path in resolved_required_filespecs_dict.keys() if a_path.resolve().exists()] + [str(a_path) for a_path in resolved_optional_filespecs_dict.keys() if a_path.resolve().exists()]
all_output_file_paths


In [ ]:

# curr_active_pipeline.register_output_file(output_path=output_path, 
curr_active_pipeline.registered_output_files_list
curr_active_pipeline.registered_output_files
# curr_active_pipeline.get_output_path()

# reveal_in_system_file_manager(curr_active_pipeline.get_daily_programmatic_session_output_path())

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadData, saveData

# Save in case something goes wrong:
temp_out_path = 'registered_output_files.pkl'


In [ ]:
saveData(temp_out_path, (curr_active_pipeline.registered_output_files.to_dict()))
reveal_in_system_file_manager(temp_out_path)

In [ ]:
test_registered_output_files = loadData(temp_out_path)
test_registered_output_files
curr_active_pipeline.registered_output_files = test_registered_output_files

# End Run

In [7]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']


In [ ]:
# Clearing 'long_short_post_decoding'
# del curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
# drop_columns = [col for col in df2.columns if col in df1.columns]
# df1 = df1.drop(drop_columns, axis=1)

# grid_bin_bounds calculation without user interaction

In [ ]:
# global_session.laps.to_dataframe()
lap_specific_epochs = curr_active_pipeline.sess.laps.as_epoch_obj().get_non_overlapping().filtered_by_duration(1.0, 30.0) # laps specifically for use in the placefields with non-overlapping, duration, constraints: the lap must be at least 1 second long and at most 30 seconds long

pos_df = global_session.position.to_dataframe()
# pos_df.time_slicer.slice(

In [ ]:

laps_pos_df = global_session.position.to_dataframe()[global_session.position.to_dataframe().lap.notnull()]
laps_only_grid_bin_bounds = PlacefieldComputationParameters.compute_grid_bin_bounds(laps_pos_df.x.to_numpy(), laps_pos_df.y.to_numpy())
print(f'laps_only_grid_bin_bounds: {laps_only_grid_bin_bounds}')

In [ ]:
unfiltered_sess_grid_bin_bounds = PlacefieldComputationParameters.compute_grid_bin_bounds(curr_active_pipeline.sess.position.x, curr_active_pipeline.sess.position.y)
print(f'unfiltered_sess_grid_bin_bounds: {unfiltered_sess_grid_bin_bounds}')
# long_pos_df, short_pos_df, global_pos_df, all_pos_df = [a_sess.position.to_dataframe() for a_sess in (long_session, short_session, global_session, curr_active_pipeline.sess)]

long_grid_bin_bounds, short_grid_bin_bounds, global_grid_bin_bounds, all_grid_bin_bounds = [PlacefieldComputationParameters.compute_grid_bin_bounds(a_sess.position.x, a_sess.position.y) for a_sess in (long_session, short_session, global_session, curr_active_pipeline.sess)]
print(long_grid_bin_bounds, short_grid_bin_bounds, global_grid_bin_bounds, all_grid_bin_bounds, sep="\n")

# 2023-06-13 - Updating Object Persistance from the dates added to the local computations:
TODO 2023-06-13 18:07: - [ ] The global_computation_results should check the `current_computation_completion_times` corresponding to the local functions that contribute results that they depend on (like 'pf1D', etc) to make sure they haven't changed. If they have changed, they should recompute.

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import LoadedObjectPersistanceState
from datetime import datetime
# curr_active_pipeline.pipeline_compare_dict
# curr_active_pipeline.computation_results['maze1']['computation_times']
	
any_most_recent_computation_time, each_epoch_latest_computation_time, each_epoch_each_result_computation_completion_times = curr_active_pipeline.get_computation_times()
each_epoch_each_result_computation_completion_times

# any_latest_computation_time: 2023-06-13 17:40:44.628404
# current_epoch_latest_computation_time: {'maze1': datetime.datetime(2023, 6, 13, 17, 39, 18, 195870), 'maze2': datetime.datetime(2023, 6, 13, 17, 40, 44, 628404), 'maze': datetime.datetime(2023, 6, 13, 17, 33, 33, 859610)}
# current_computation_status: {'maze1': {'_perform_baseline_placefield_computation': datetime.datetime(2023, 6, 13, 17, 24, 17, 681574),
#   '_perform_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 39, 18, 195870),
#   '_perform_two_step_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 26, 17, 278340),
#   '_perform_pf_find_ratemap_peaks_computation': datetime.datetime(2023, 6, 13, 17, 26, 21, 5364),
#   '_perform_extended_statistics_computation': datetime.datetime(2023, 6, 13, 17, 26, 21, 31363),
#   '_perform_firing_rate_trends_computation': datetime.datetime(2023, 6, 13, 17, 26, 22, 385595)},
#  'maze2': {'_perform_baseline_placefield_computation': datetime.datetime(2023, 6, 13, 17, 26, 23, 665116),
#   '_perform_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 40, 44, 628404),
#   '_perform_two_step_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 28, 32, 360685),
#   '_perform_pf_find_ratemap_peaks_computation': datetime.datetime(2023, 6, 13, 17, 28, 36, 25350),
#   '_perform_extended_statistics_computation': datetime.datetime(2023, 6, 13, 17, 28, 36, 49351),
#   '_perform_firing_rate_trends_computation': datetime.datetime(2023, 6, 13, 17, 28, 37, 302445)},
#  'maze': {'_perform_baseline_placefield_computation': datetime.datetime(2023, 6, 13, 17, 28, 39, 869814),
#   '_perform_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 31, 56, 800175),
#   '_perform_two_step_position_decoding_computation': datetime.datetime(2023, 6, 13, 17, 33, 26, 857989),
#   '_perform_pf_find_ratemap_peaks_computation': datetime.datetime(2023, 6, 13, 17, 33, 30, 796529),
#   '_perform_extended_statistics_computation': datetime.datetime(2023, 6, 13, 17, 33, 30, 836528),
#   '_perform_firing_rate_trends_computation': datetime.datetime(2023, 6, 13, 17, 33, 33, 859610)}}



In [ ]:
curr_active_pipeline.active_completed_computation_result_names

In [ ]:
curr_active_pipeline.pipeline_compare_dict

In [ ]:

# ('computed_data', ('pf1D', 'pf2D', 'pf1D_Decoder', 'pf2D_Decoder', 'pf2D_TwoStepDecoder', 'RatemapPeaksAnalysis', 'extended_stats', 'firing_rate_trends'))


# 2023-05-26 - Expected vs Observed firing rates
## 2023-05-30 Comments from Kourosh:
- concern: is it valid to use the most-likely predicted position for each decoder for this? Isn't it a bit circular because the firing rates determine the likelihood of the position?
- sign-rank is equivalent of paired t-test for non-parametric, use that to compare the means for each cell after averaging over replays to remove the noise from that
- OPTION: could add a continuous plot that shows the difference between the two decoders and the animal's measured position as a function of time (observed_x - long_decoded_x), (observed_x - short_decoded_x).
- [x] Plot just replay-to-replay series, removing the time axis

In [ ]:
## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

num_epochs: int = len(num_timebins_in_epoch)

## 2023-05-30 - `plot_expected_vs_observed`

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _prepare_plot_expected_vs_observed

fig, axes, final_context, active_out_figure_paths = _prepare_plot_expected_vs_observed(curr_active_pipeline, defer_render=False)

In [ ]:
## 2023-05-31 - Reference Output of matplotlib figure to file, along with building appropriate context.
active_session_figures_out_path = curr_active_pipeline.get_daily_programmatic_session_output_path()
final_context = curr_active_pipeline.sess.get_context().adding_context('display_fn', display_fn_name='plot_expected_vs_observed').adding_context('display_kwargs', **display_kwargs)
print(f'final_context: {final_context}')
active_out_figure_paths = perform_write_to_file(fig, final_context, figures_parent_out_path=active_session_figures_out_path, register_output_file_fn=curr_active_pipeline.register_output_file)


In [ ]:
all_epochs_computed_observed_from_expected_difference_LONG[0] #.shape (42, 8)

In [ ]:
all_epochs_computed_observed_from_expected_difference_maximum_LONG[0] # .shape (42, 8)

In [ ]:
all_epochs_computed_expected_cell_firing_rates_mean_LONG[0] # .shape (42,)

In [ ]:
all_epochs_computed_observed_from_expected_difference_LONG[0].shape

In [ ]:
jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

In [ ]:
jonathan_firing_rate_analysis_result.rdf.rdf

In [ ]:
jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins #.shape

In [ ]:
import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
instantaneous_fr_df = deepcopy(jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.instantaneous_unit_specific_spike_rate_values)
instantaneous_fr_df['t'] = jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins
instantaneous_fr_df #.plot()

smoothed_instantaneous_fr_df = instantaneous_fr_df.ewm(span=60).mean()

smoothed_instantaneous_fr_df.plot(x='t')
plt.title("Instantaneous Firing Rates")
plt.xlabel('time')
plt.ylabel('Firing Rate (Hz)')

In [ ]:
instantaneous_fr_df.set_index('t').to_clipboard()

In [ ]:
## Require placefield presence on either the long or the short
jonathan_firing_rate_analysis_result.neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])]

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])].index.to_numpy()

# 2023-05-19 - Testing S-only emergence, L-only replays in S, peak position remappings 

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df.copy()
neuron_replay_stats_df

In [ ]:
neuron_replay_stats_df.sort_values(by=['long_pf_peak_x'], inplace=False, ascending=True)

In [ ]:

from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
## 2023-05-19 - Get S-only pfs
is_S_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_long_pf']), neuron_replay_stats_df['has_short_pf'])
is_S_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY
assert (is_S_pf_only == is_S_only).all()
S_only_aclus = neuron_replay_stats_df.index[is_S_only].to_numpy()
neuron_replay_stats_df[is_S_pf_only]


In [ ]:

## Show L-only pfs stop replaying on S
is_L_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_short_pf']), neuron_replay_stats_df['has_long_pf'])
is_L_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY
assert (is_L_pf_only == is_L_only).all()
L_only_aclus = neuron_replay_stats_df.index[is_L_only].to_numpy()
neuron_replay_stats_df[is_L_only]


## For ('kdiba', 'gor01', 'one', '2006-6-09_1-22-43') - Have L-only cells [24, 98] that have ['short_num_replays'] = [8, 7]. We were hoping that there would be few to no replays on the S-track that involved L-only cells.

In [ ]:
## 2023-05-23 - Get Common (SHARED) placefields
## Goal 1: From the cells with the placefields on both tracks, compute the degree to which they remap in position and sort them according to their distance.
is_BOTH_pf_only = np.logical_and(neuron_replay_stats_df['has_short_pf'], neuron_replay_stats_df['has_long_pf']) # (63,)
BOTH_pf_only_aclus = neuron_replay_stats_df.index[is_BOTH_pf_only].to_numpy()

## NOTE: is_BOTH_pf_only is a much more stringent requirement (and a strict subset) than `is_BOTH_only`
is_BOTH_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.SHARED # (99,)
BOTH_only_aclus = neuron_replay_stats_df.index[is_BOTH_only].to_numpy()
assert BOTH_only_aclus.shape[0] >= BOTH_pf_only_aclus.shape[0]

BOTH_pf_only_df = neuron_replay_stats_df[is_BOTH_pf_only].copy()
BOTH_pf_only_df['long_short_pf_peak_x_displacement'] = BOTH_pf_only_df['long_pf_peak_x'].values - BOTH_pf_only_df['short_pf_peak_x'].values
BOTH_pf_only_df['long_short_pf_peak_x_distance'] = BOTH_pf_only_df['long_short_pf_peak_x_displacement'].abs()
BOTH_pf_only_df.sort_values(by=['long_short_pf_peak_x_distance'], inplace=True, ascending=False)
BOTH_pf_only_df

#TODO 2023-05-23 - Can do more detailed peaks analysis with: long_results.RatemapPeaksAnalysis and short_results.RatemapPeaksAnalysis



In [ ]:
""" 24 cells, 133 Epochs, 789 Total Timebins
# Other
	
	- is_non_firing_time_bin: numpy.ndarray - (24, 789)
	- all_epochs_num_epoch_time_bins: numpy.ndarray - (133,)
	
    ## Indexing Helpers:
		- all_epochs_reverse_flat_epoch_indicies_array: numpy.ndarray - (789,)
		- split_by_epoch_reverse_flattened_time_bin_indicies: list - (133,)
    
	- original_1D_decoder: pyphoplacecellanalysis.Analysis.Decoder.reconstruction.BasePositionDecoder
		- pf: neuropy.analyses.placefields.PfND
		- neuron_IDXs: numpy.ndarray - (24,)
		- neuron_IDs: numpy.ndarray - (24,)
		- F: numpy.ndarray - (119, 24)
		- P_x: numpy.ndarray - (119, 1)
        
	- flat_all_epochs_decoded_epoch_time_bins: numpy.ndarray - (24, 789)

# Measured
	- flat_all_epochs_measured_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - (24, 789)


# Expected
	- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_firing_rates: numpy.ndarray - (24, 789)
    
    
    
## Epoch-based
	- all_epochs_decoded_epoch_time_bins_mean: numpy.ndarray - (133, 24)

    
    
    
- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - .shape: (24, 789) 
- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - .shape: (24, 789)


"""

In [ ]:
long_results_obj.flat_all_epochs_computed_expected_cell_firing_rates
long_results_obj.flat_all_epochs_measured_cell_firing_rates

# long_results_obj.result_df_grouped

measured_dfs = pd.DataFrame(long_results_obj.flat_all_epochs_measured_cell_firing_rates.T, columns=long_results_obj.original_1D_decoder.neuron_IDs)
expected_dfs = pd.DataFrame(long_results_obj.flat_all_epochs_computed_expected_cell_firing_rates.T, columns=long_results_obj.original_1D_decoder.neuron_IDs)
# pd.DataFrame(np.stack((long_results_obj.flat_all_epochs_measured_cell_firing_rates, long_results_obj.flat_all_epochs_computed_expected_cell_firing_rates)), columns=['measured_fr', 'expected_fr'])
measured_dfs.plot()


In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df.to_clipboard()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import compute_evening_morning_parition

difference_sorted_aclus, evening_sorted_aclus, morning_sorted_aclus = compute_evening_morning_parition(neuron_replay_stats_df, debug_print=True)
sorted_neuron_replay_stats_df = neuron_replay_stats_df.reindex(difference_sorted_aclus).copy() # This seems to work to re-sort the dataframe by the sort indicies
sorted_neuron_replay_stats_df

In [ ]:
from neuropy.core.neurons import NeuronType

%matplotlib qt
# Look at replays during ripples vs. those not during ripples. Also potentially PBEs vs. not PBEs.

# NeuronType.from_any_string_series(['pyr','intr'])
'pyr','cont','intr'

active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
curr_active_pipeline.display('_display_jonathan_interactive_replay_firing_rate_comparison', active_identifying_session_ctx, included_neuron_types=NeuronType.from_any_string_series(['pyr'])) # only the pyramidal neurons

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
curr_active_pipeline.display('_display_jonathan_interactive_replay_firing_rate_comparison', active_identifying_session_ctx, included_neuron_types=NeuronType.from_any_string_series(['intr']), require_placefield=False) # only the pyramidal neurons

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import build_replays_custom_scatter_markers, CustomScatterMarkerMode

_curr_included_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])].index.to_numpy()
_curr_output = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', active_identifying_session_ctx, included_unit_neuron_IDs=_curr_included_aclus, marker_split_mode=CustomScatterMarkerMode.NoSplit)

## 2023-04-13 - Shuffled Surprise
""" 
Relevant Functions:
`perform_full_session_leave_one_out_decoding_analysis`:
	`perform_leave_one_aclu_out_decoding_analysis`:	from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import perform_leave_one_aclu_out_decoding_analysis
	`_analyze_leave_one_out_decoding_results`: from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.DefaultComputationFunctions import _analyze_leave_one_out_decoding_results
"""


In [ ]:
# 2023-04-18 - Refactored into decoder_result
result, result_df, result_df_grouped = long_results_obj.result, long_results_obj.result_df, long_results_obj.result_df_grouped

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:

# 1. Get set of cells active in a given time bin, for each compute the surprise of its placefield with the leave-one-out decoded posterior.

# 2. From the remainder of cells (those not active), randomly choose one to grab the placefield of and compute the surprise with that and the same posterior.

# Expectation: The cells that are included in the time bin are expected to have a lower surprise (be less correlated with) the posterior.

In [ ]:
win, plots = plot_long_short_surprise_difference_plot(curr_active_pipeline, long_results_obj, short_results_obj, long_epoch_name, short_epoch_name)


In [ ]:
long_results_obj.timebinned_neuron_info.n_timebins # 736
short_results_obj.timebinned_neuron_info.n_timebins

In [ ]:
long_results_obj.flat_all_epochs_measured_cell_firing_rates.shape # (30, 736)
# short_results_obj.flat_all_epochs_measured_cell_firing_rates.shape # (30, 736)



In [ ]:
win, plots_tuple, legend = plot_long_short_expected_vs_observed_firing_rates(long_results_obj, short_results_obj, limit_aclus=list(BOTH_pf_only_df.index[:2].values))
long_epoch_indicator_region_items, short_epoch_indicator_region_items = _helper_add_long_short_session_indicator_regions(win, long_epoch=curr_active_pipeline.filtered_epochs[long_epoch_name], short_epoch=curr_active_pipeline.filtered_epochs[short_epoch_name])

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot # works pretty well seemingly

export_pyqtgraph_plot(win)
# pg.setConfigOption('leftTitle', 'MathText')
# win.setTitle(r'JSD(p_x_given_n, pf[<font size="4"><b><span style="color:red;">i</span></b></font>]) - JSD(p_x_given_n, pf[<font size="4"><b>j</b></font>]) where <font size="4"><b>j</b></font> non-firing')

# win.setTitle(generate_html_string(f'JSD(p_x_given_n, pf[{i_str}]) - JSD(p_x_given_n, pf[{j_str}]) where {j_str} non-firing', font_size=8))

# r'$\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$'
# title_str

## Pre 2023-04-13

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt

from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure

from neuropy.core.neurons import NeuronType
# # Include only pyramidal aclus:
# print(f'all shared_aclus: {len(shared_aclus)}\nshared_aclus: {shared_aclus}')
# shared_aclu_neuron_type = long_session.neurons.neuron_type[np.isin(long_session.neurons.neuron_ids, shared_aclus)]
# assert len(shared_aclu_neuron_type) == len(shared_aclus)
# # Find only the aclus that are pyramidal:
# is_shared_aclu_pyramidal = (shared_aclu_neuron_type == NeuronType.PYRAMIDAL)
# pyramidal_only_shared_aclus = shared_aclus[is_shared_aclu_pyramidal]
# print(f'num pyramidal_only_shared_aclus: {len(pyramidal_only_shared_aclus)}\npyramidal_only_shared_aclus: {pyramidal_only_shared_aclus}')


## Drop Pyramidal but don't use only shared aclus:
all_aclus = deepcopy(long_session.neurons.neuron_ids)
neuron_type = long_session.neurons.neuron_type
assert len(neuron_type) == len(all_aclus)
# Find only the aclus that are pyramidal:
is_aclu_pyramidal = (neuron_type == NeuronType.PYRAMIDAL)
pyramidal_only_all_aclus = all_aclus[is_aclu_pyramidal]
print(f'num pyramidal_only_all_aclus: {len(pyramidal_only_all_aclus)}\npyramidal_only_all_aclus: {pyramidal_only_all_aclus}')

is_aclu_interneuron = (neuron_type == NeuronType.INTERNEURONS)
interneuron_only_all_aclus = all_aclus[is_aclu_interneuron]
print(f'num interneuron_only_all_aclus: {len(interneuron_only_all_aclus)}\ninterneurons: {interneuron_only_all_aclus}')

# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, shared_aclus, epoch_idx=5, callout_epoch_IDXs=[0,1,2,3], skip_rendering_callouts=True)
# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_shared_aclus, epoch_idx=2, callout_epoch_IDXs=[0,4], skip_rendering_callouts=False)
app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=6, callout_epoch_IDXs=[2,4,6], skip_rendering_callouts=False)

In [ ]:

# Use the Jupyter Index Thing
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:
active_fig_obj.export()

In [ ]:
app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=11, callout_epoch_IDXs=[0,1,2, 3, 4, 5], skip_rendering_callouts=False)

# 2023-04-13 - Find Good looking epochs:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.patheffects as path_effects

from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

curr_results_obj = long_results_obj
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_pagination_controller = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(curr_results_obj.active_filter_epochs, curr_results_obj.all_included_filter_epochs_decoder_result, 
	xbin=curr_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='TestDecodedEpochSlicesPaginationController', active_context=active_identifying_session_ctx,  max_subplots_per_page=200) # 10
# _out_pagination_controller

### 2023-05-30 - Add the radon-transformed linear fits to each epoch to the stacked epoch plots:

In [ ]:
from neuropy.utils.result_context import IdentifyingContext
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import plot_decoded_epoch_slices_paginated

_out_pagination_controller, active_out_figure_paths, final_context = plot_decoded_epoch_slices_paginated(curr_active_pipeline, long_results_obj, curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj'), save_figure=True)


In [ ]:
_out_pagination_controller, active_out_figure_paths, final_context = plot_decoded_epoch_slices_paginated(curr_active_pipeline, short_results_obj, curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj'), save_figure=True)


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _prepare_plot_long_and_short_epochs

(pagination_controller_L, pagination_controller_S), (fig_L, fig_S), (ax_L, ax_S), (final_context_L, final_context_S), (active_out_figure_paths_L, active_out_figure_paths_S) = _prepare_plot_long_and_short_epochs(curr_active_pipeline, defer_render=False)

# Other Programmatic Figures

In [ ]:
curr_active_pipeline.registered_output_files
# curr_active_pipeline.registered_output_files_list

In [ ]:
curr_active_pipeline.reload_default_display_functions()
active_identifying_session_ctx, active_session_figures_out_path, active_out_figures_list = batch_programmatic_figures(curr_active_pipeline)
batch_extended_programmatic_figures(curr_active_pipeline=curr_active_pipeline)

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [8]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons

## Builds proper sort indicies for '_display_short_long_pf1D_comparison'
long_ratemap = long_pf1D.ratemap
short_ratemap = short_pf1D.ratemap

curr_any_context_neurons = _find_any_context_neurons(*[k.neuron_ids for k in [long_ratemap, short_ratemap]])
curr_any_context_neurons

array([  2,   3,   7,   8,   9,  11,  12,  15,  17,  19,  22,  23,  24,
        25,  29,  31,  34,  36,  37,  38,  39,  41,  42,  44,  46,  47,
        50,  51,  54,  56,  57,  59,  60,  61,  63,  64,  65,  66,  67,
        68,  69,  70,  71,  74,  75,  76,  77,  78,  82,  83,  85,  86,
        89,  91,  92,  94,  95,  96,  97, 100, 101, 103])

## Sorting Helpers:

In [10]:
def _sort_desired(extant_arr, desired_sort_arr):
    """ 
    Want to find the set of sort indicies that can be applied to extant_arr s.t.
    (extant_arr[out_sort_idxs] == desired_sort_arr)
    
    INEFFICIENT: O^n^2
    
    Usage:
    
        new_all_aclus_sort_indicies = _sort_desired(active_2d_plot.neuron_ids, all_sorted_aclus)
        assert len(new_all_aclus_sort_indicies) == len(active_2d_plot.neuron_ids), f"need to have one new_all_aclus_sort_indicies value for each neuron_id"
        assert np.all(active_2d_plot.neuron_ids[new_all_aclus_sort_indicies] == all_sorted_aclus), f"must sort "
        new_all_aclus_sort_indicies
    """
    missing_aclu_indicies = np.isin(extant_arr, desired_sort_arr, invert=True)
    missing_aclus = extant_arr[missing_aclu_indicies] # array([ 3,  4,  8, 13, 24, 34, 56, 87])
    if len(missing_aclus) > 0:
        desired_sort_arr = np.concatenate((desired_sort_arr, missing_aclus)) # the final desired output order of aclus. Want to compute the indicies that are required to sort an ordered array of indicies in this order
        ## TODO: what about entries in desired_sort_arr that might be missing in extant_arr?? Hopefully never happens.
    assert len(desired_sort_arr) == len(extant_arr), f"need to have one all_sorted_aclu value for each neuron_id but len(desired_sort_arr): {len(desired_sort_arr)} and len(extant_arr): {len(extant_arr)}"
    # sort_idxs = np.array([desired_sort_arr.tolist().index(v) for v in extant_arr])
    sort_idxs = np.array([extant_arr.tolist().index(v) for v in desired_sort_arr])
    assert len(sort_idxs) == len(extant_arr), f"need to have one new_all_aclus_sort_indicies value for each neuron_id"
    assert np.all(extant_arr[sort_idxs] == desired_sort_arr), f"must sort: extant_arr[sort_idxs]: {extant_arr[sort_idxs]}\n desired_sort_arr: {desired_sort_arr}"
    return sort_idxs, desired_sort_arr

In [12]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df.copy()

In [17]:
test_df = neuron_replay_stats_df.sort_values(by=['long_pf_peak_x'], inplace=False, ascending=True).copy()
test_df = test_df[np.isin(test_df.index, curr_any_context_neurons)]
test_df

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
92,18.018588,True,52.018588,True,False,SplitPartitionMembership.SHARED,1.212647,0.890202,-0.322446,10.762744,10.067359,-0.695385,5.987695,5.478780,-0.508915,90,25,14,11,NeuronType.PYRAMIDAL
91,18.018588,True,52.018588,True,False,SplitPartitionMembership.SHARED,1.644951,0.837320,-0.807632,14.282971,11.762396,-2.520575,7.963961,6.299858,-1.664103,89,20,13,7,NeuronType.PYRAMIDAL
47,18.018588,True,198.018588,True,False,SplitPartitionMembership.SHARED,0.993312,1.384729,0.391417,60.393507,10.309588,-50.083918,30.693409,5.847158,-24.846251,45,12,1,11,NeuronType.PYRAMIDAL
101,18.018588,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,2.718215,0.919096,-1.799119,18.019961,19.150225,1.130264,10.369088,10.034660,-0.334427,99,33,19,14,NeuronType.PYRAMIDAL
54,36.018588,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.417598,0.238529,-0.179069,11.865386,7.415215,-4.450171,6.141492,3.826872,-2.314620,52,4,2,2,NeuronType.PYRAMIDAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,240.018588,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,2.357836,0.976005,-1.381831,31.461754,28.210910,-3.250843,16.909795,14.593458,-2.316337,34,8,2,6,NeuronType.PYRAMIDAL
19,242.018588,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.578695,0.359582,-0.219113,35.395542,37.108055,1.712513,17.987118,18.733819,0.746700,17,5,1,4,NeuronType.PYRAMIDAL
89,242.018588,True,50.018588,True,False,SplitPartitionMembership.SHARED,2.501139,1.699489,-0.801651,16.938301,13.102549,-3.835752,9.719720,7.401019,-2.318701,87,38,16,22,NeuronType.PYRAMIDAL
8,NaN,False,194.018588,True,True,SplitPartitionMembership.RIGHT_ONLY,0.466354,2.154300,1.687945,11.839761,23.905534,12.065773,6.153058,13.029917,6.876859,6,8,3,5,NeuronType.PYRAMIDAL


In [15]:
## Sorts aclus using `neuron_replay_stats_df`'s columns (  neuron_replay_stats_df = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']['neuron_replay_stats_df'] -- this is produced by '_perform_jonathan_replay_firing_rate_analyses') :
_sorted_neuron_stats_df = neuron_replay_stats_df.sort_values(by=["long_pf_peak_x", "short_pf_peak_x", 'neuron_IDX'], ascending=[True, True, True]).copy()
_sorted_neuron_stats_df = _sorted_neuron_stats_df[np.isin(_sorted_neuron_stats_df.index, curr_any_context_neurons)] # clip to only those neurons included in `curr_any_context_neurons`
_sorted_aclus = _sorted_neuron_stats_df.index.to_numpy()
_sorted_neuron_IDXs = _sorted_neuron_stats_df.neuron_IDX.to_numpy()
print(f'_sorted_aclus: {_sorted_aclus}')
print(f'_sorted_neuron_IDXs: {_sorted_neuron_IDXs}')

## Use this sort for the 'curr_any_context_neurons' sort order:
new_all_aclus_sort_indicies, desired_sort_arr = _sort_desired(curr_any_context_neurons, _sorted_aclus)
print(f'new_all_aclus_sort_indicies: {new_all_aclus_sort_indicies}')
new_all_aclus_sort_indicies

_sorted_aclus: [ 91  92  47 101  63  25  54  42  69  60  31  41  11  68  46   2  65  78
  86  59  12   7  74 103  64  76 100  44  24  61  34  56  39   9  85  38
  22  51  15  66  23  57  75  71  95  37  94  96  97  77  17  70  50  67
   3  83  29  36  89  19  82   8]
_sorted_neuron_IDXs: [ 89  90  45  99  61  23  52  40  67  58  29  39   9  66  44   0  63  76
  84  57  10   5  72 101  62  74  98  42  22  59  32  54  37   7  83  36
  20  49  13  64  21  55  73  69  93  35  92  94  95  75  15  68  48  65
   1  81  27  34  87  17  80   6]
new_all_aclus_sort_indicies: [53 54 25 60 34 13 28 22 40 32 15 21  5 39 24  0 36 47 51 31  6  2 43 61
 35 45 59 23 12 33 16 29 20  4 50 19 10 27  7 37 11 30 44 42 56 18 55 57
 58 46  8 41 26 38  1 49 14 17 52  9 48  3]


array([53, 54, 25, 60, 34, 13, 28, 22, 40, 32, 15, 21,  5, 39, 24,  0, 36,
       47, 51, 31,  6,  2, 43, 61, 35, 45, 59, 23, 12, 33, 16, 29, 20,  4,
       50, 19, 10, 27,  7, 37, 11, 30, 44, 42, 56, 18, 55, 57, 58, 46,  8,
       41, 26, 38,  1, 49, 14, 17, 52,  9, 48,  3])

In [ ]:
# ## Sorts aclus using 'pf1D' linear peak position order:
# long_results = curr_active_pipeline.computation_results['maze1_PYR'].computed_data
# short_results = curr_active_pipeline.computation_results['maze2_PYR'].computed_data
# all_results = curr_active_pipeline.computation_results['maze_PYR'].computed_data

# pf1D_neuronIDs = np.array(all_results.pf1D.ratemap.neuron_ids)
# pf1D_sort = all_results.pf1D.ratemap.get_sort_indicies()
# print(f'pf1D_neuronIDs: {pf1D_neuronIDs}')
# print(f'pf1D_sort: {pf1D_sort}')
# pf1D_sorted_neuronIDs = pf1D_neuronIDs[pf1D_sort]
# print(f'pf1D_sorted_neuronIDs: {pf1D_sorted_neuronIDs}')
# # pf1D_neuronIDs, pf1D_sort, pf1D_sorted_neuronIDs

# ## Use this sort for the 'active_2d_plot' sort order:
# new_all_aclus_sort_indicies, desired_sort_arr = _sort_desired(active_2d_plot.neuron_ids, pf1D_sorted_neuronIDs)
# print(f'new_all_aclus_sort_indicies: {new_all_aclus_sort_indicies}')
# new_all_aclus_sort_indicies

In [ ]:
# missing_neuron_ids = np.array(list(set(curr_any_context_neurons) - set(long_ratemap.neuron_ids))) # neuron_ids that we won't have an active_curve for
# sentinal_max_value = np.iinfo(missing_neuron_ids.dtype).max # the value to stand in effectively for infinity so that it's always sorted alst in the np.argsort(...) call.
# active_curve_peak_locations = np.argmax(long_ratemap.normalized_tuning_curves, axis=1) # the index of the peak, one for each neuron
# active_curve_peak_locations = np.hstack([active_curve_peak_locations, np.full_like(missing_neuron_ids, sentinal_max_value)]) # add one infinite entry for each of the missing_neuron_ids
# sort_ind = np.argsort(active_curve_peak_locations) # sort_ind = np.argsort(np.argmax(long_ratemap.normalized_tuning_curves, axis=1))
# sort_ind

In [ ]:
# long_ratemap.normalized_tuning_curves.shape # (36, 112)
# # long_ratemap.normalized_tuning_curves[:, active_curve_peak_locations] #.shape # (36, 36)

# [long_ratemap.normalized_tuning_curves[i, active_curve_peak_locations[i]] for i in np.arange(len(active_curve_peak_locations))]

In [18]:
# Given a `sorted_list`, a list previously sorted by some criteria (such as its corresponding values in a dataframe column) and `reference_list`, a list containing the same elements in their initial order to serve as a reference => find the transformation indicies such that `sorted_list = reference_list[sort_indicies]`

def find_transformation_indices(sorted_list, reference_list):
    """ Given a `sorted_list`, a list previously sorted by some criteria (such as its corresponding values in a dataframe column) and `reference_list`, a list containing the same elements in their initial order to serve as a reference
        => find the transformation indicies such that `sorted_list = reference_list[sort_indicies]`
    Example:
        sorted_list = np.array([4, 6, 2])
        reference_list = np.array([2, 4, 6])
        sort_indicies = np.array([1, 2, 0])
    """
    # Convert to NumPy arrays if the inputs are not already arrays
    if not isinstance(sorted_list, np.ndarray):
        sorted_array = np.array(sorted_list)
    else:
        sorted_array = sorted_list

    if not isinstance(reference_list, np.ndarray):
        reference_array = np.array(reference_list)
    else:
        reference_array = reference_list

    # Use argsort to get the indices that would sort the reference_array
    # sort_indices = np.argsort(reference_array) # does not work
    
    sort_indices = []
    for element in sorted_array:
        index = np.searchsorted(reference_array, element)
        sort_indices.append(index)
        
    sort_indices = np.array(sort_indices)
    # assert (reference_array[sort_indicies] == sorted_array).all(), f"Testing the sort_indicies did not work! reference_array[sort_indices]: {reference_array[sort_indices]} but it should equal {sorted_array}"
    return sort_indices

# sort_indicies = find_transformation_indices(sorted_list = [4, 6, 2], reference_list = [2, 4, 6])
# sort_indicies

# # Apply the sort_indices to the sorted_array
# test_reconstructed_sorted_list = reference_list[sort_indicies]
# test_reconstructed_sorted_list
# assert (test_reconstructed_sorted_list == sorted_list).all()

In [19]:
sort_indicies = find_transformation_indices(sorted_list=test_df.index.to_numpy(), reference_list=curr_any_context_neurons)
sort_indicies # close to `new_all_aclus_sort_indicies` but strangely not exact.

array([54, 53, 25, 60, 28, 13, 34, 32, 40, 22, 21, 15,  5, 39, 24,  0, 36,
       47, 31, 51,  6,  2, 43, 61, 35, 45, 59, 23, 12, 16, 33, 29, 20,  4,
       50, 19, 10, 27,  7, 37, 11, 30, 44, 42, 18, 56, 55, 57, 58, 46,  8,
       41, 26, 38,  1, 49, 14, 17,  9, 52,  3, 48])

In [20]:
sort_indicies.shape

(62,)

In [ ]:
test_reconstructed_sorted_list = curr_any_context_neurons[sort_indicies]
test_reconstructed_sorted_list
# assert (test_reconstructed_sorted_list == test_df.index.to_numpy()).all()

In [ ]:
sort_indicies

In [ ]:
curr_any_context_neurons[sort_ind]

In [ ]:
sort_ind = np.argsort(test_df.index) # sort_ind = np.argsort(np.argmax(long_ratemap.normalized_tuning_curves, axis=1))
sort_ind

In [21]:
from neuropy.plotting.ratemaps import plot_ratemap_1D
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_short_v_long_pf1D_comparison, LongShortTrackComparingDisplayFunctions

%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None
# sort_idx = None
# sort_idx = sort_ind
# sort_idx = sort_indicies
sort_idx = new_all_aclus_sort_indicies

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=True, debug_print=True, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )

# ax = out.axes[0]

include_includelist: ['maze1', 'maze2', 'maze']
long_epoch_name: maze1, short_epoch_name: maze2, global_epoch_name: maze
n_neurons: 62
shared_fragile_neuron_IDXs: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61].	 np.shape: (62,)
curr_any_context_neurons: [  2   3   7   8   9  11  12  15  17  19  22  23  24  25  29  31  34  36
  37  38  39  41  42  44  46  47  50  51  54  56  57  59  60  61  63  64
  65  66  67  68  69  70  71  74  75  76  77  78  82  83  85  86  89  91
  92  94  95  96  97 100 101 103].	 np.shape: (62,)
reuse_axs_tuple is None. Making new figures/axes
included_unit_neuron_IDs: [  2   3   7   8   9  11  12  15  17  19  22  23  24  25  29  31  34  36
  37  38  39  41  42  44  46  47  50  51  54  56  57  59  60  61  63  64
  65  66  67  68  69  70  71  74  75  76  77  78  82  83  85  86  89  91
  92  94  95  96  97 100 101 103]
	 n_

In [ ]:
curr_active_pipeline.display('_display_short_long_pf1D_comparison', single_figure=False)

In [ ]:

sort_idx = np.argsort(product_overlap_scalars_df.prod_overlap.to_numpy())[::-1] # the `[::-1]` term reverses the array, which by defaul is returned in ascending order and we want descending
sort_idx

In [ ]:
active_curves_dict = dict(zip(long_ratemap.neuron_ids, long_ratemap.normalized_tuning_curves))


In [27]:
from neuropy.plotting.ratemaps import plot_ratemap_1D

In [ ]:

sortby = np.arange(len(new_all_aclus_sort_indicies)) # just lienarly ascending
# sortby = new_all_aclus_sort_indicies # set correct sort indicies
# included_unit_neuron_IDs = curr_any_context_neurons # no pre-sort
included_unit_neuron_IDs = curr_any_context_neurons[new_all_aclus_sort_indicies] # pre-sort
_out = plot_ratemap_1D(long_pf1D.ratemap, normalize_tuning_curve=True, included_unit_indicies=None, included_unit_neuron_IDs=included_unit_neuron_IDs, sortby=sortby)

## FINALLY WORKING for ordering the placefields themselves by pre-sorting the included_unit_neuron_IDs. Abolish the redundant fields for this plotting function.

In [30]:
# sortby = np.arange(len(new_all_aclus_sort_indicies)) # just lienarly ascending
sortby = new_all_aclus_sort_indicies # set correct sort indicies
included_unit_neuron_IDs = curr_any_context_neurons # no pre-sort
# included_unit_neuron_IDs = curr_any_context_neurons[new_all_aclus_sort_indicies] # pre-sort
_out = plot_ratemap_1D(long_pf1D.ratemap, normalize_tuning_curve=True, included_unit_indicies=None, included_unit_neuron_IDs=included_unit_neuron_IDs, sortby=sortby)

In [26]:
plot_ratemap_1D(short_pf1D.ratemap, normalize_tuning_curve=True, included_unit_indicies=None, included_unit_neuron_IDs=included_unit_neuron_IDs, sortby=sortby)

(<Axes: title={'center': '1D Placemaps Placemaps (54 good cells)'}, xlabel='Position'>,
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61]),
 array([[0.51764706, 0.57794118, 0.63823529, 0.69852941, 0.75882353,
         0.81911765, 0.87941176, 0.93970588, 1.        , 0.69803922,
         0.73578431, 0.77352941, 0.81127451, 0.84901961, 0.88676471,
         0.9245098 , 0.9622549 , 1.        , 0.54901961, 0.60539216,
         0.66176471, 0.71813725, 0.7745098 , 0.83088235, 0.8872549 ,
         0.94362745, 1.        , 0.38823529, 0.44845291, 0.51024503,
         0.57361166, 0.63855281, 0.70506846, 0.77315863, 0.84282331,
         0.9140625 , 0.22352941, 0.2725518 , 0.32637032, 0.38498496,
         0.44839572, 0.51660261, 0.58960561, 0.66740475, 0.75      ,
         

# 2023-06-01 - Testing new metrics:

In [ ]:

curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
## Extract variables from results object:
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']


print(f'long_test: {np.mean(long_long_diff)}')
print(f'short_test: {np.mean(short_short_diff)}')

In [ ]:
long_session.replay

In [ ]:
# long_results_obj.original_1D_decoder
replay_result_df = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())
replay_result_df

In [ ]:
short_results_obj.active_filter_epochs.to_dataframe()

In [ ]:
# Look at lap speed over time - doesn't look like there's a difference between the long and the short track speeds
global_session.position.compute_higher_order_derivatives()
running_pos_df = global_session.position.to_dataframe()
ax = running_pos_df.plot(x='t', y=['lin_pos','speed'], title='Running Speed over Time')
ax

In [ ]:
# Call the function
fig, ax = plot_lines(replay_result_df)

In [ ]:
import matplotlib
%matplotlib qt

curr_active_pipeline.reload_default_display_functions()
out_man = curr_active_pipeline.get_output_manager()
out_man.get_figure_output_path
curr_active_pipeline.write_figure_to_output_path



In [ ]:
_out = curr_active_pipeline.display('_display_grid_bin_bounds_validation')
# _display_running_and_replay_speeds_over_time


# 2023-06-13 11:01: - [ ] New display function testing

In [7]:
import matplotlib as mpl
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path, programmatic_display_to_PDF
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_pdf_metadata_from_display_context # newer version of build_pdf_export_metadata
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

%matplotlib qt

from pyphoplacecellanalysis.General.Mixins.ExportHelpers import programmatic_render_to_file

In [ ]:
curr_active_pipeline.reload_default_display_functions()


In [8]:
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap, visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_raster_plot # used in `plot_kourosh_activity_style_figure`
import pyphoplacecellanalysis.External.pyqtgraph as pg # used in `plot_kourosh_activity_style_figure`

heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(long_pf1D.ratemap.normalized_tuning_curves, title=f"pf1D Visualization")
# Apply the colormap
# heatmap_pf1D_img.setLookupTable(lut)
heatmap_pf1D_win.show()

qt.qpa.wayland: setGrabPopup called with a parent, QtWaylandClient::QWaylandXdgSurface(0xf34aef0) which does not match the current topmost grabbing popup, QtWaylandClient::QWaylandXdgSurface(0x141ff710) According to the xdg-shell protocol, this is not allowed. The wayland QPA plugin is currently handling it by setting the parent to the topmost grabbing popup. Note, however, that this may cause positioning errors and popups closing unxpectedly because xdg-shell mandate that child popups close before parents
qt.qpa.wayland: setGrabPopup called with a parent, QtWaylandClient::QWaylandXdgSurface(0xf34aef0) which does not match the current topmost grabbing popup, QtWaylandClient::QWaylandXdgSurface(0x141ff710) According to the xdg-shell protocol, this is not allowed. The wayland QPA plugin is currently handling it by setting the parent to the topmost grabbing popup. Note, however, that this may cause positioning errors and popups closing unxpectedly because xdg-shell mandate that child popu

In [ ]:
heatmap_pf1D_win.close()

# Work on converting `active_config.plotting_config` and `active_config.computation_config` to IdentifyingContext

In [ ]:
active_config = curr_active_pipeline.active_configs['maze1']
plotting_config = active_config.plotting_config
computation_config = active_config.computation_config
pf_params: PlacefieldComputationParameters = computation_config.pf_params
spike_analysis: DynamicContainer = computation_config.spike_analysis
session_config = active_config.active_session_config

In [ ]:
# computation_config.to_dict()

pf_params_context = IdentifyingContext.init_from_dict(pf_params.to_dict())
pf_params_context

In [ ]:
spike_analysis_context = IdentifyingContext.init_from_dict(spike_analysis.to_dict())
spike_analysis_context

In [ ]:
# not quite right:
computation_config_context = IdentifyingContext(pf_params=pf_params_context, spike_analysis=spike_analysis_context)
computation_config_context

In [ ]:
plotting_config.get_figure_save_path()

In [ ]:
#TODO 2023-06-14 07:49: - [ ] Conceptual Arguments for specifying how to display a dict-like:
keys_transformer=lambda k: k.to_lower()
values_transformer=lambda v: np.array(v) # get precision and stuff